In [8]:
from sklearn.cluster import AgglomerativeClustering
from src.utils import load_json
from collections import defaultdict
import pandas as pd

In [5]:
rxns = load_json("../data/sprhea/v3_folded_test.json")
rxns = {int(k) : v for k, v in rxns.items()}

In [13]:
features = ["smarts", "rcs", "min_rules"]
data = defaultdict(list)
for k, v in rxns.items():
    data["id"].append(k)
    for feat in features:
        data[feat].append(v[feat])

X = pd.DataFrame(
    data=data
)

X.head()

,id,smarts,rcs,min_rules
0,1760,C[S+](CCCN)CC1OC(n2cnc3c(N)ncnc32)C(O)C1O.O=C=...,"[[[4], [1, 0]], [[4, 6, 8]]]","[rule0023, rule0023]"
1,404,NC(=O)OP(=O)(O)O.NCCCC(N)C(=O)O>>NC(=O)NCCCC(N...,"[[[1, 3], [0]], [[1, 3], [2]]]","[rule0098, rule0098]"
2,174,NC(=O)NC(CC(=O)O)C(=O)O>>O=C1CC(C(=O)O)NC(=O)N1.O,"[[[0, 6, 8]], [[1, 10], [0]]]","[rule0106, rule0106]"
3,128,C[S+](CCC(N)C(=O)O)CC1OC(n2cnc3c(N)ncnc32)C(O)...,"[[[0, 1], [9]], [[14], [10, 9]]]","[rule0011, rule0011]"
4,743,NC(=O)C1=CN(C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4c...,"[[[3, 4, 5, 41, 42, 43], [1, 3]], [[1, 3], [3,...","[rule0003, rule0003]"


In [ ]:
cutoff = -0.9
ac = AgglomerativeClustering(
    metric=rcmcs_dist,
    distance_threshold=cutoff
)